# Hive assignment 1

The purpose of this task is to create an external table on the posts data of the `stackoverflow.com` website.

## Step 4. Complete the assignment

In [1]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;


-- Create external table 

-- Your code here
CREATE EXTERNAL TABLE posts_sample_external (
    Id string,
    CreationYear string,
    CreationMonth string
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+).*\")(?=.*\\bCreationDate=\"(\\d{4}-\\d+).*\").*' 
    )
LOCATION '/data/stackexchange1000/posts';

Overwriting task1_create_external_table.hql


Make sure that you have created your table correctly. Select the first 10 posts from the dataset.

In [2]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here
SELECT * FROM demodb.posts_sample_external
LIMIT 10;

Overwriting task1_check_select.hql


Create managed table `posts_sample`. Create the partition by the month and by the year. 

In [3]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    Id string
)
PARTITIONED BY (
    CreationMonth string,
    CreationYear string
);


Overwriting task1_create_managed_table.hql


Populate data from the table `posts_sample_external` to the table `posts_sample`. Don't forget about the partitioning rules!

In [4]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table

USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.enforce.bucketing=true;
SET hive.mapred.supports.subdirectories=true;

-- Your code here for inserting data
FROM posts_sample_external pse
INSERT overwrite table posts_sample
PARTITION (CreationMonth, CreationYear)
SELECT pse.Id, pse.CreationMonth, pse.CreationYear;



Overwriting task1_insert_table.hql


Make sure that your table contains appropriate data about posts

In [5]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT * FROM demodb.posts_sample
LIMIT 10;

Overwriting task1_watch_new_table.hql


Take the third row of the dataset in the ascending order for the posts (firstly by year, after that by month)

In [6]:
%%writefile task1_result.hql
-- Your code here
with cte as (
    select count(*) as total, CreationYear, CreationMonth
    from demodb.posts_sample
    group by CreationYear, CreationMonth
), cte2 as (
    select total, CreationYear, CreationMonth,
        dense_rank() over (order by CreationMonth) as rang
    from cte
)
select CreationYear, CreationMonth, total
from cte2
where rang=3;


Overwriting task1_result.hql


## Step 5. Submission part. Do not touch!! And simple run all cells below!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!

In [7]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

Take a look at your submission query!

In [8]:
!cat task1.hql

-- Create external table posts_sample_external with suitable values
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;


-- Create external table 

-- Your code here
CREATE EXTERNAL TABLE posts_sample_external (
    Id string,
    CreationYear string,
    CreationMonth string
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+).*\")(?=.*\\bCreationDate=\"(\\d{4}-\\d+).*\").*' 
    )
LOCATION '/data/stackexchange1000/posts';-- create managed table
-- Check that this table contains info about year and month
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parc

In [9]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [10]:
%%bash
hive -f task1.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.096 seconds
OK
Time taken: 0.124 seconds
OK
Time taken: 0.888 seconds
OK
Time taken: 0.023 seconds
OK
Time taken: 0.019 seconds
OK
Time taken: 0.276 seconds
OK
Time taken: 0.018 seconds
Query ID = jovyan_20190515121818_ae92e52e-2d30-4cf8-b073-e1ca103ad600
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1557922020218_0007, Tracking URL = http://89a08070e8e4:8088/proxy/application_1557922020218_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1557922020218_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2019-05-15 12:18:18,909 Stage-1 map = 0%,  reduce = 0%
2019-05-15 12:18:37,426 Stage-1 map = 39%,  reduce = 0%, Cumulative CPU 16.68 sec
2019-05-15 12:18:42,782 Stage-1 map = 62%,  reduce = 0%, Cumulative CPU 22.36 

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!